In [ ]:
!pip install tensorflow==2.11.0 opencv-python mediapipe sklearn matplotlib
import cv2
import mediapipe as mp
import numpy as np
import os
import tensorflow as tf
from google.colab.patches import cv2_imshow

### Preprocess data, add labels

In [4]:
actions = np.array(['fidgety','cramped_synchronized'])

In [5]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [6]:
#create a dictionary
label_map = {label:num for num, label in enumerate(actions)}

In [7]:
DATA_PATH = '/content/drive/MyDrive/Keypoints data/MP_data' 
actions = np.array(['cramped_synchronized','fidgety'])

In [82]:
sequence_length = 120

In [112]:
sequences, labels = [], []
no_sequences = 53 
action = 'cramped_synchronized'
for sequence in range(no_sequences):
  window =[]
  for frame_num in range (sequence_length):
    res = np.load(os.path.join(DATA_PATH, action, str(sequence), str(frame_num) + '.npy'))
    window.append(res)
  sequences.append(window)
  labels.append(label_map[action])

In [84]:
no_sequences = 53
action = 'fidgety'
for sequence in range(no_sequences):
  window =[]
  for frame_num in range (sequence_length):
    res = np.load(os.path.join(DATA_PATH, action, str(sequence), str(frame_num) + '.npy'))
    window.append(res)
  sequences.append(window)
  labels.append(label_map[action])

In [ ]:
#/content/drive/MyDrive/Keypoints data/MP_data/cramped_synchronized
#/content/drive/MyDrive/Keypoints data/MP_data/fidgety

In [85]:
np.array(sequences).shape


(134, 120, 132)

In [86]:
X = np.array(sequences)

In [87]:
X.shape


(134, 120, 132)

In [88]:
y = to_categorical(labels).astype(int)

In [ ]:
y

In [92]:
np.array(labels).shape

(134,)

In [93]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size =0.05)

In [96]:
y_test.shape

(7, 2)

### Build and train LSTM Neural Network

In [99]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [102]:
#create log directory and set up tensorflow callback
log_dir = os.path.join('Logs')
tb_callback = TensorBoard (log_dir = log_dir)

In [104]:
model = Sequential()
model.add(LSTM(64,return_sequences=True, activation='relu', input_shape=(120,132)))
model.add(LSTM(128,return_sequences=True, activation='relu')) #true because there is another layer next
model.add(LSTM(64,return_sequences=False, activation='relu')) # false because this is the last layer
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [105]:
model.compile(optimizer='Adam', loss = 'categorical_crossentropy', metrics=['categorical_accuracy'])

In [109]:
#train the model
model.fit(x_train,y_train, epochs = 15, callbacks=[tb_callback])

Epoch 1/15
4/4 [==============================] - 1s 330ms/step - loss: nan - categorical_accuracy: 0.5906
Epoch 2/15
4/4 [==============================] - 1s 332ms/step - loss: nan - categorical_accuracy: 0.5906
Epoch 3/15
4/4 [==============================] - 2s 416ms/step - loss: nan - categorical_accuracy: 0.5906
Epoch 4/15
4/4 [==============================] - 2s 609ms/step - loss: nan - categorical_accuracy: 0.5906
Epoch 5/15
4/4 [==============================] - 2s 605ms/step - loss: nan - categorical_accuracy: 0.5906
Epoch 6/15
4/4 [==============================] - 2s 598ms/step - loss: nan - categorical_accuracy: 0.5906
Epoch 7/15
4/4 [==============================] - 2s 379ms/step - loss: nan - categorical_accuracy: 0.5906
Epoch 8/15
4/4 [==============================] - 1s 335ms/step - loss: nan - categorical_accuracy: 0.5906
Epoch 9/15
4/4 [==============================] - 1s 325ms/step - loss: nan - categorical_accuracy: 0.5906
Epoch 10/15
4/4 [====================

In [110]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 120, 64)           50432     
                                                                 
 lstm_4 (LSTM)               (None, 120, 128)          98816     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 2)                 66        
                                                                 
Total params: 204,962
Trainable params: 204,962
Non-tr

In [111]:
model.predict(x_test)

1/1 [==============================] - 1s 594ms/step


array([[nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan]], dtype=float32)